In [17]:
import gym
import math
import time
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
np.random.seed(0)

In this notebook all algorithms that I reference are from the textbook: Reinforcement Learning: An Introduction by Sutton and Barto.

https://github.com/openai/gym/wiki/CartPole-v0

This link has all the information regarding the specifics of the CartPole environment, such as the action space, how rewards are disitributed, and what it means to solve this problem.

Solved requirements: "Considered solved when the average reward is greater than or equal to 195.0 over 100 consecutive trials." 

## Q-Learning

We will first attempt to solve this problem with the Q-learning algorithm, which is an off-policy temporal difference control algorithm.

In [92]:
# Create the environment with openai gym and set a seed to work in
env = gym.make('CartPole-v1')
env.seed(0)

[0]

In [93]:
n_bins = (6, 12, 6 , 12)
lower_bounds = [-4.8, env.observation_space.low[1], env.observation_space.low[2], -42]
upper_bounds = [4.8, env.observation_space.high[0], env.observation_space.high[2], 42]

def discretizer(pos, cart_velocity, angle, pole_velocity):
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds])
    return tuple(map(int,est.transform([[pos, cart_velocity, angle, pole_velocity]])[0]))

In [94]:
Q_table = np.zeros(n_bins + (env.action_space.n,))
Q_table.shape

(6, 12, 6, 12, 2)

In [152]:
def qlearn():
    tic = time.time()
    reward_list = []
    Q_table = np.zeros(n_bins + (env.action_space.n,))
    total_reward = 0
    consec_trials = 0
    for i in range(1, 10001):
        current, done = discretizer(*env.reset()), False
        while not done:
            lr = max(0.01, min(1.0, 1.0 - math.log10((i + 1) / 25)))
            epsilon =  max(0.05, min(1, 1.0 - math.log10((i  + 1) / 25)))
            if np.random.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(Q_table[current])
            observation, reward, done, info = env.step(action)
            next_state = discretizer(*observation)
            Q_table[current][action] += lr*(reward + np.max(Q_table[next_state]) - Q_table[current][action])
            current = next_state
            total_reward += reward
        if i % 100 == 0:
            print(total_reward)
            reward_list.append(total_reward/100)
            for index in range(len(reward_list)):
                if reward_list[index] >= 195.0:
                    toc = time.time()
                    first_avg_r = index
                    return f'Took {toc-tic:.2f} seconds and achieved average reward of {reward_list[first_avg_r]} in {(first_avg_r+1)*100} episodes'
            total_reward = 0
    toc = time.time()
    print(f'Took {toc-tic:.2f} seconds and failed')
    env.close()

In [153]:
qlearn()

2586.0
3726.0
3652.0
2782.0
2999.0
3341.0
3089.0
3340.0
3467.0
3284.0


KeyboardInterrupt: 

In [154]:
def sarsa():
    tic = time.time()
    reward_list = []
    Q_table = np.zeros(n_bins + (env.action_space.n,))
    total_reward = 0
    consec_trials = 0
    for i in range(1, 10001):
        current, done = discretizer(*env.reset()), False
        while not done:
            lr = max(0.01, min(1.0, 1.0 - math.log10((i + 1) / 25)))
            epsilon =  max(0.05, min(1, 1.0 - math.log10((i  + 1) / 25)))
            if np.random.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(Q_table[current])
            observation, reward, done, info = env.step(action)
            next_state = discretizer(*observation)
            Q_table[current][action] += lr*(reward + Q_table[next_state][np.argmax(Q_table[next_state])] - Q_table[current][action])
            current = next_state
            total_reward += reward
        if i % 100 == 0:
            print(total_reward)
            reward_list.append(total_reward/100)
            for index in range(len(reward_list)):
                if reward_list[index] >= 195.0:
                    toc = time.time()
                    first_avg_r = index
                    return f'Took {toc-tic:.2f} seconds and achieved average reward of {reward_list[first_avg_r]} in {(first_avg_r+1)*100} episodes'
            total_reward = 0
    toc = time.time()
    print(f'Took {toc-tic:.2f} seconds and failed')
    env.close()

In [155]:
sarsa()

2247.0
2645.0
4323.0
4150.0
4299.0
4434.0
4218.0
4448.0
4335.0
4361.0
3999.0
4512.0
4449.0
4708.0
4174.0
4544.0
4951.0
4458.0
4261.0
3886.0
3829.0
4197.0
4387.0
4754.0
4265.0
4596.0
4140.0
4235.0
4660.0
4337.0
4526.0
4152.0
3961.0
4234.0
4779.0
4522.0
4370.0
4742.0
4314.0
4399.0
4344.0
3908.0
4345.0
4386.0
4212.0


KeyboardInterrupt: 